### Anatomy registration to ephys 
#### This code is designed to guide you through registering an anatmap and applying those channel locations to the data We assume that by now, you have registered the brain to the atlas, traced the tracks and put them into the correct folder in histology, with the correct name format. 

#### now, you need to run this code in "iblenv". executing the first cell will allow you to call a desired anatmap and will format the data correctly for the IBL gui to use. This formatting takes substantial time. 


In [9]:
# add a func to loop over SC recordings (selected somehow by Flora)
import json,glob,sys,datetime
import pandas as pd 
import numpy as np
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
from Processing.pykilo.convert_to_ibl_format import add_anat_to_ibl_format
from Admin.csv_queryExp import queryCSV
from Processing.pyhist.assign_clusters_to_atlas import call_for_anatmap_recordings,save_out_cluster_location,save_to_common_anatmap
#session_list = pd.read_csv(prPath / 'Helpers/sc_selected_recordings.csv')
#session_list = queryCSV(subject='FT009',expDate='2021-01-20',expDef='sparseNoise',checkSpikes='1')
probe = 'probe1'
session_list = call_for_anatmap_recordings(subject='AV025',checkSpikes='1',probe=probe,depth_selection='auto') 
for _,r in session_list.iterrows():
    # read corresponding ephys files 
    corresponding_ephys_json= list((Path(r.expFolder)).glob('ONE_preproc\probe*\_av_rawephys.path*.json'))
    for rec_path in corresponding_ephys_json: 
            # open json to get the actual path
            rec = open(rec_path,)
            rec = json.load(rec)
            rec = Path(rec)
            print(rec.parents[2])
            s=add_anat_to_ibl_format(rec.parents[2],recompute=False)


\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\ephys\AV025_2022-11-08_SparseNoise_Part1_g0\AV025_2022-11-08_SparseNoise_Part1_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\ephys\AV025_2022-11-08_SparseNoise_Part1_g0\AV025_2022-11-08_SparseNoise_Part1_g0_imec1
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part2_g0\AV025_2022-11-07_SparseNoise_Part2_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\ephys\AV025_2022-11-07_SparseNoise_Part2_g0\AV025_2022-11-07_SparseNoise_Part2_g0_imec1
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\ephys\AV025_2022-11-08_SparseNoise_Part2_g0\AV025_2022-11-08_SparseNoise_Part2_g0_imec0
success in saving xyz_picks from track.
\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\ephys\AV025_2022-11-08_SparseNoise_Part2_g0\AV025_2022-11-08_SparseNoise_Part2

### Good job. Now you are ready to use the gui. Print session_list with the below cell and register all your recordings to ephys!
#### cheat to gui:
###### conda activate iblenv
###### cd C:\Users\Flora\Documents\Github\int-brain-lab\iblapps\atlaselectrophysiology
###### python ephys_atlas_gui.py -o True

#### tip: if you don't know which ones you have done already, proceed to the next cell and come back.

In [10]:
session_list


,expDate,expNum,expDef,expDuration,rigName,existBlock,existTimeline,existFrontCam,existSideCam,existEyeCam,...,extractEvents,expFolder,ephysPathProbe0,ephysPathProbe1,subject,probe1_raw,shank_range,depth_range,shank,botrow
4,2022-11-08,3,sparseNoise,729,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\3,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,AV025,"{'channels': {'localCoordinates': [[0. 0.], [3...","(0.0, 32.0)","(0.0, 2865.0)",0,0
1,2022-11-07,5,sparseNoise,744,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\5,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025,{'channels': {'localCoordinates': [[ 0. 2880...,"(0.0, 32.0)","(2880.0, 5745.0)",0,192
5,2022-11-08,4,sparseNoise,745,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\4,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,AV025,{'channels': {'localCoordinates': [[ 200. 2160...,"(200.0, 232.0)","(0.0, 2865.0)",1,0
2,2022-11-07,6,sparseNoise,669,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\6,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025,{'channels': {'localCoordinates': [[ 200. 5040...,"(200.0, 232.0)","(2880.0, 5745.0)",1,192
6,2022-11-08,5,sparseNoise,729,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\5,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,AV025,{'channels': {'localCoordinates': [[400. 720.]...,"(400.0, 432.0)","(0.0, 2865.0)",2,0
3,2022-11-07,7,sparseNoise,663,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\7,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025,{'channels': {'localCoordinates': [[ 400. 3600...,"(400.0, 432.0)","(2880.0, 5745.0)",2,192
7,2022-11-08,6,sparseNoise,687,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08\6,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-08...,AV025,{'channels': {'localCoordinates': [[ 600. 1440...,"(600.0, 632.0)","(0.0, 2865.0)",3,0
0,2022-11-07,1,sparseNoise,672,zelda-stim3,1,1,1,1,1,...,1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025,{'channels': {'localCoordinates': [[ 600. 4320...,"(600.0, 632.0)","(2880.0, 5745.0)",3,192


### Done? Was it tedious...? Well, the hard part is over. Now just run the below code, such that we save a rough channel map 

In [13]:
last_date = np.max(session_list.expDate.values)
for _,r in session_list.iterrows():
    one_path = Path(r.expFolder)  / 'ONE_preproc' / probe
    save_out_cluster_location(one_path) # for the recordings in question
    save_to_common_anatmap(one_path,probe,r.shank,r.botrow,last_date)

## now we apply the contstructed anatmaps for the rest of the data 

In [14]:
# get all recordings 
all_rec_list = queryCSV(subject=r.subject,expDate='postImplant')
all_rec_list = all_rec_list.dropna(subset='extractSpikes')
all_rec_list = all_rec_list[[rec.extractSpikes[int(probe[-1])*2]=='1' for _,rec in all_rec_list.iterrows()]]
# for each recording search for nearest anatmap available 
anatmap_paths = Path(all_rec_list.iloc[0].expFolder).parents[1]
anatmap_paths = anatmap_paths / 'histology/registered_anatmaps'


for _,r in all_rec_list.iterrows(): 
    my_expDate = datetime.datetime.strptime(r.expDate,'%Y-%m-%d')
    anatmap_dates = [datetime.datetime.strptime(d.name,'%Y-%m-%d') for d in list(anatmap_paths.iterdir())]
    timediffs = [abs(d-my_expDate) for d in anatmap_dates]
    selected_anatmap_date = anatmap_dates[np.argmin(timediffs)]
    selected_anatmaps = list((anatmap_paths / selected_anatmap_date.strftime('%Y-%m-%d')).glob('*%s*.json' % probe))
    one_path = Path(r.expFolder)  / 'ONE_preproc' / probe
    save_out_cluster_location(one_path,anatmap_paths=selected_anatmaps)

In [7]:
all_rec_list

,expDate,expNum,expDef,expDuration,rigName,existBlock,existTimeline,existFrontCam,existSideCam,existEyeCam,...,fMapFrontCam,fMapSideCam,fMapEyeCam,issortedPyKS,extractSpikes,extractEvents,expFolder,ephysPathProbe0,ephysPathProbe1,subject
0,2022-11-07,1,sparseNoise,672,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025
1,2022-11-07,2,multiSpaceWorld_checker_training,5792,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\2,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025
3,2022-11-07,4,AVPassive_ckeckerboard_postactive,3461,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\4,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025
4,2022-11-07,5,sparseNoise,744,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\5,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025
5,2022-11-07,6,sparseNoise,669,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07\6,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,\\zaru.cortexlab.net\Subjects\AV025\2022-11-07...,AV025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2022-12-02,3,sparseNoise,684,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02\3,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,AV025
134,2022-12-02,5,imageWorld_AllInOne,865,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02\5,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,AV025
135,2022-12-02,7,sparseNoise,668,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02\7,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,AV025
136,2022-12-02,8,sparseNoise,662,zelda-stim3,1,1,1,1,1,...,1,1,1,"1,1","1,1",1,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02\8,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,\\zaru.cortexlab.net\Subjects\AV025\2022-12-02...,AV025
